# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 9:24AM,255071,19,TR-RA-01172023,"GCH Granules USA, Inc.",Released
1,Jan 16 2023 9:21AM,255060,10,0086298657,ISDIN Corporation,Released
2,Jan 16 2023 9:21AM,255060,10,0086298953,ISDIN Corporation,Released
3,Jan 16 2023 9:21AM,255060,10,0086298955,ISDIN Corporation,Released
4,Jan 16 2023 9:21AM,255060,10,0086298963,ISDIN Corporation,Released
5,Jan 16 2023 9:21AM,255060,10,0086298961,ISDIN Corporation,Released
6,Jan 16 2023 9:21AM,255060,10,0086298967,ISDIN Corporation,Released
7,Jan 16 2023 9:21AM,255060,10,0086298968,ISDIN Corporation,Released
8,Jan 16 2023 9:21AM,255060,10,0086298966,ISDIN Corporation,Released
9,Jan 16 2023 9:21AM,255060,10,0086298971,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,255067,Released,4
12,255068,Released,2
13,255069,Released,18
14,255070,Released,1
15,255071,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Released
id,
255067,4
255068,2
255069,18
255070,1
255071,1


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Released
id,
255053,1
255054,7
255055,1
255056,1
255057,20


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,Released
id,
255053,1
255054,7
255055,1
255056,1
255057,20


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Released
0,255053,1
1,255054,7
2,255055,1
3,255056,1
4,255057,20


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,id,Released
0,255053,1
1,255054,7
2,255055,1
3,255056,1
4,255057,20


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 9:24AM,255071,19,"GCH Granules USA, Inc."
1,Jan 16 2023 9:21AM,255060,10,ISDIN Corporation
40,Jan 16 2023 9:20AM,255070,20,"HVL, LLC dba Atrium Innovations"
41,Jan 16 2023 9:20AM,255069,10,"Methapharm, Inc."
59,Jan 16 2023 9:16AM,255068,10,"Snap Medical Industries, LLC"
61,Jan 16 2023 9:15AM,255067,10,Emerginnova
65,Jan 16 2023 9:15AM,255065,10,Methapharm-G
67,Jan 16 2023 9:15AM,255065,10,"Methapharm, Inc."
77,Jan 16 2023 9:14AM,255063,12,"SD Head USA, LLC"
78,Jan 16 2023 9:12AM,255061,16,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Released
0,Jan 16 2023 9:24AM,255071,19,"GCH Granules USA, Inc.",1
1,Jan 16 2023 9:21AM,255060,10,ISDIN Corporation,39
2,Jan 16 2023 9:20AM,255070,20,"HVL, LLC dba Atrium Innovations",1
3,Jan 16 2023 9:20AM,255069,10,"Methapharm, Inc.",18
4,Jan 16 2023 9:16AM,255068,10,"Snap Medical Industries, LLC",2
5,Jan 16 2023 9:15AM,255067,10,Emerginnova,4
6,Jan 16 2023 9:15AM,255065,10,Methapharm-G,12
7,Jan 16 2023 9:15AM,255065,10,"Methapharm, Inc.",12
8,Jan 16 2023 9:14AM,255063,12,"SD Head USA, LLC",1
9,Jan 16 2023 9:12AM,255061,16,"Emersa Waterbox, LLC",1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released
0,Jan 16 2023 9:24AM,255071,19,"GCH Granules USA, Inc.",1
1,Jan 16 2023 9:21AM,255060,10,ISDIN Corporation,39
2,Jan 16 2023 9:20AM,255070,20,"HVL, LLC dba Atrium Innovations",1
3,Jan 16 2023 9:20AM,255069,10,"Methapharm, Inc.",18
4,Jan 16 2023 9:16AM,255068,10,"Snap Medical Industries, LLC",2
5,Jan 16 2023 9:15AM,255067,10,Emerginnova,4
6,Jan 16 2023 9:15AM,255065,10,Methapharm-G,12
7,Jan 16 2023 9:15AM,255065,10,"Methapharm, Inc.",12
8,Jan 16 2023 9:14AM,255063,12,"SD Head USA, LLC",1
9,Jan 16 2023 9:12AM,255061,16,"Emersa Waterbox, LLC",1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Jan 16 2023 9:24AM,255071,19,"GCH Granules USA, Inc.",1
1,Jan 16 2023 9:21AM,255060,10,ISDIN Corporation,39
2,Jan 16 2023 9:20AM,255070,20,"HVL, LLC dba Atrium Innovations",1
3,Jan 16 2023 9:20AM,255069,10,"Methapharm, Inc.",18
4,Jan 16 2023 9:16AM,255068,10,"Snap Medical Industries, LLC",2


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released
0,Jan 16 2023 9:24AM,255071,19,"GCH Granules USA, Inc.",1
1,Jan 16 2023 9:21AM,255060,10,ISDIN Corporation,39
2,Jan 16 2023 9:20AM,255070,20,"HVL, LLC dba Atrium Innovations",1
3,Jan 16 2023 9:20AM,255069,10,"Methapharm, Inc.",18
4,Jan 16 2023 9:16AM,255068,10,"Snap Medical Industries, LLC",2


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Jan 16 2023 9:24AM,255071,19,"GCH Granules USA, Inc.",1
1,Jan 16 2023 9:21AM,255060,10,ISDIN Corporation,39
2,Jan 16 2023 9:20AM,255070,20,"HVL, LLC dba Atrium Innovations",1
3,Jan 16 2023 9:20AM,255069,10,"Methapharm, Inc.",18
4,Jan 16 2023 9:16AM,255068,10,"Snap Medical Industries, LLC",2


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released
Warehouse,,
10,2295563,96
12,765178,22
16,255061,1
19,510126,2
20,255070,1
22,255053,1


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released
0,10,2295563,96
1,12,765178,22
2,16,255061,1
3,19,510126,2
4,20,255070,1
5,22,255053,1


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released
0,10,96
1,12,22
2,16,1
3,19,2
4,20,1
5,22,1


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,96
1,12,Released,22
2,16,Released,1
3,19,Released,2
4,20,Released,1


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,22
Status,,,,,,
Released,96,22,1,2,1,1


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,22
0,Released,96,22,1,2,1,1


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,22
0,Released,96,22,1,2,1,1


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()